In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Dateien einlesen
idn_df = pd.read_excel("idn.xlsx", dtype=str)  # IDNs laden
keywords_df = pd.read_excel("schlagworte_systematik_gewichtung.xlsx", dtype=str)  # Schlagwörter mit Gewichtung + Systematik

# Schlagwörter vorbereiten
keywords = keywords_df["Schlagwort"].dropna().tolist()
weights = keywords_df.set_index("Schlagwort")["Gewichtung"].astype(int).to_dict()
systematics = keywords_df.set_index("Schlagwort")["Systematik"].to_dict()
alone_status = keywords_df.set_index("Schlagwort")["Alleinstehend"].fillna("").to_dict() if "Alleinstehend" in keywords_df.columns else {}

# Funktion zum Abrufen und Durchsuchen der Texte
def find_keywords(idn):
    url_text = f"https://d-nb.info/{idn}/04/text"
    url_pdf = f"https://d-nb.info/{idn}/04/pdf"
    try:
        response = requests.get(url_text, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text().lower()
            
            word_counts = {}  # Speichert Schlagwort: (Anzahl, Gewicht, Systematik)
            
            for word in keywords:
                pattern = re.compile(rf"\b\w*{re.escape(word.lower())}\w*\b", re.IGNORECASE)
                matches = pattern.findall(text)
                count = len(matches)
                
                if count > 0:
                    weighted = count * weights.get(word, 1)
                    word_counts[word] = (count, weighted, systematics.get(word, ""))
            
            # Berechnung der Gesamtsumme der Gewichtungen (nur unique Schlagwörter)
            total_weight = sum(weights.get(word, 1) for word in word_counts.keys())

            # Entscheidungskriterien
            if len(word_counts) >= 3 or total_weight >= 5:
                formatted = [
                    f"{word} ({count} Treffer, Gewicht={weights.get(word,1)}, Score={weighted}, Systematik={systematics.get(word,'')})"
                    for word, (count, weighted, _) in word_counts.items()
                ]
                return url_text, url_pdf, "; ".join(formatted), total_weight

            elif len(word_counts) == 1:
                word, (count, weighted, syst) = list(word_counts.items())[0]
                if count >= 2 and alone_status.get(word, "").strip().lower() == "x":
                    return url_text, url_pdf, f"{word} ({count} Treffer, Gewicht={weights.get(word,1)}, Score={weighted}, Systematik={syst})", total_weight
    except requests.RequestException:
        return url_text, url_pdf, "Fehler", 0
    
    return url_text, url_pdf, None, 0

# Neue Spalten füllen
idn_df[["URL", "PDF-URL", "Gefundene Schlagwörter", "Gesamtgewichtung"]] = idn_df.iloc[:, 0].apply(find_keywords).apply(pd.Series)

# Nur Zeilen mit gefundenen Schlagwörtern behalten
idn_df = idn_df.dropna(subset=["Gefundene Schlagwörter"])

# Spaltennamen setzen
idn_df.rename(columns={idn_df.columns[0]: "IDN"}, inplace=True)

# Ergebnis speichern
if not idn_df.empty:
    idn_df.to_excel("idn_mit_schlagwort_gewichtung.xlsx", index=False)
    print("Die Datei 'idn_mit_schlagwort_gewichtung.xlsx' wurde erfolgreich erstellt!")
else:
    print("Keine passenden Schlagwörter gefunden – es wurde keine Datei erstellt.")


C:\Users\sickerti\AppData\Local\Temp\ipykernel_10008\3834971614.py:23: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, "html.parser")


Die Datei 'idn_mit_schlagwort_gewichtung.xlsx' wurde erfolgreich erstellt!
